In [1]:
# Imports
import csv
import glob
import json
import pandas as pd

In [2]:
# Input the path to the cluster tsv file
cluster_path = '/home/jovyan/work/single_cell/week_13_no_batch_correction/gene_for_gsea_list.tsv'

# If there are multiple gene sets that are not already combined, add there
# path to the strings in the list. The final results will be a 
# merged gene set model for fgsea. 
gene_set_path = ['go_heart.gmt', 'KEGG_CELL_Cycle.gmt', 'ex.gmt']

In [3]:
def load_gene_set(gene_set_path):
    """
    Load the gene set model(s) and generate a dictionary containing the pathway name,
    description, and genes. Generate a union set of all genes that are found in the 
    provided gene model(s).
    
    :param gene_set_path: A list of string(s) that are the path(s) to the gene set model(s)
    
    return:
    gene_sets: A dictionary where the keys are the pathway name and the values
    are a list. The first item in the list is the pathway description and the
    second item is the set of genes for that pathway. 
    
    total: A set containing the union of all genes in the gene model(s)
    """
    gene_sets = dict()
    total = set()
    
    for paths in gene_set_path:
        # If the file is present open and generate dictionary and union set
        if glob.glob(paths):
            with open(paths) as file:
                for line in file:
                    line = line.strip().split('\t')
                    set_name = line[0]
                    set_description = line[1]
                    gene_subset = set(line[2:])
                    gene_sets[set_name] = [set_description, gene_subset]
                    for items in line[2:]:
                        total.add(items)
        # If the file is not present then print an error message to the screen
        else:
            print("The path provided does not contain geneset: {}".format(paths))
    print("Loaded %d gene sets" % len(gene_sets))
    print("Total number of genes: {}".format(len(total)))   
    return gene_sets, total

In [4]:
def load_cluster(cluster_path):
    """
    Load cluster output file into a pandas dataframe
    
    :param cluster_path: A string that is the path to the cluster file
    
    return:
    cluster_df: A Pandas DataFrame containing the clusters and z-score ranked genes
    """
    # If the path to the cluster is present, then load the file into a pandas dataframe
    if glob.glob(cluster_path):
        cluster_df = pd.read_csv(cluster_path, sep='\t')
        cluster_df = cluster_df.set_index('cluster_number')
        return cluster_df
    # If the file is not present, then print an error to the screen
    else:
        print("The path provided does not contain cluster output file!")

In [5]:
def intersection(cluster_df, total):
    """
    Compute intersecting genes between the clusters and gene model(s)
    
    :param cluster_df: A Pandas DataFrame containing the clusters and z-score ranked genes
    :param total: A set containing the union of all genes in the gene model(s)
    
    return: None
    """
    # Generate a list of cluster numbers
    cluster_list = list(set(cluster_df.index.tolist()))
    # Subset the first cluster from cluster_df
    cluster_zero = cluster_df.loc[cluster_list[0]]
    # Obtain the genes in that first cluster.
    # NOTE: This assumes that the same genes
    # are present in the total current cluster file
    cluster_zero_genes = set(cluster_zero['gene'])
    # Obtain the intersecting genes
    number_of_interesecting = cluster_zero_genes.intersection(total)
    
    print("Total number of unique genes in genesets: {}".format(len(number_of_interesecting)))
    print("Intersecting genes:")
    print(number_of_interesecting)

In [6]:
def output_combined_gmt_file(gene_set_path, gene_dict):
    """
    Generate a combined output .gmt file to the local directory
    
    :param gene_set_path: A list of string(s) that are the path(s) to the gene set model(s)
    :param gene_dict: A dictionary where the keys are the pathway name and the values
    are a list. The first item in the list is the pathway description and the
    
    return: None
    """
    # Check if the provided gene model(s) are greater than one.
    # If the condition is met then output a combined *.gmt file.
    if len(gene_set_path) > 1:
        with open ('combined_genesets.gmt', "a") as f:
            for key, value in gene_dict.items():
                f.write("{}\t{}\t{}\n".format(key,value[0], '\t'.join(map(str,value[1]))))

In [7]:
def main(cluster_path, gene_set_path):
    """
    First, generate a union set of all genes that are found in the 
    provided gene model(s), as well as a dictionary containing 
    the pathway name, description, and gene set. Second, generate
    a Pandas Dataframe containing the clusters and their gene
    z-scores. Third, compute the intersection of genes between the set
    of all genes that are found in the provided gene model(s) and
    the clusters. Finally, generate a combined .gmt file if the
    number of provided models exceed one.
    
    :param cluster_path: A string that is the path to the cluster file
    :param gene_set_path: A list of string(s) that are the path(s) to the gene set model(s)
    
    return: None
    """
    gene_sets, total = load_gene_set(gene_set_path)
    cluster_df = load_cluster(cluster_path)
    intersection(cluster_df, total)
    output_combined_gmt_file(gene_set_path, gene_sets)

In [8]:
### Executable Cell ###
main(cluster_path, gene_set_path)

The path provided does not contain geneset: ex.gmt
Loaded 46 gene sets
Total number of genes: 821
Total number of unique genes in genesets: 209
Intersecting genes:
{'SCN5A', 'GADD45G', 'CASQ2', 'ADAMTS1', 'TEK', 'COL5A1', 'GADD45B', 'CDKN2D', 'CDC20', 'SMAD6', 'TTN', 'HSPB7', 'MYLK', 'HEY2', 'HPGD', 'CCNA2', 'ADAM15', 'IRX4', 'PDE4B', 'AVPR1A', 'SEMA3C', 'SLC8A3', 'PTGER4', 'FLRT3', 'ACE', 'ALPK3', 'MYH6', 'BMP2', 'VEGFA', 'SMYD1', 'HEY1', 'LOXL1', 'SOX18', 'HOPX', 'ITPR2', 'CHRM2', 'CSRP3', 'RDH10', 'FREM2', 'PDLIM5', 'LEFTY1', 'HHEX', 'IRX5', 'FOXC2', 'PITX2', 'CDKN1A', 'BASP1', 'PLK1', 'NRG1', 'ERBB4', 'XIRP2', 'MAD2L1', 'PAM', 'RYR2', 'MYBPC3', 'ADAMTS6', 'FHOD3', 'RBM20', 'MEF2C', 'SEMA3A', 'NKX2-5', 'ABCC9', 'CORIN', 'KCND3', 'S1PR1', 'AKAP6', 'MYL2', 'CACNA1G', 'PHOX2B', 'SHOX2', 'CALCRL', 'DKK1', 'CLDN5', 'SNAI2', 'JAG1', 'MYL3', 'PLXNA4', 'ATP1A2', 'CPE', 'TAC1', 'CRIP1', 'HTR2B', 'DES', 'TBX5', 'SNTA1', 'ATP1B1', 'CCND1', 'HOXA3', 'ERG', 'ECE1', 'SORBS2', 'AGT', 'CCNB1', 'XIR